### STEP 1 Unzip 10K reports from data packages in a folder

In [1]:
# import os
# import shutil
# # The destination directory where you want to move the .txt files
# destination_dir = '/Users/chengyu/Documents/Corporate Finance/10K'

# # Making sure the destination directory exists, if not, create it
# if not os.path.exists(destination_dir):
#     os.makedirs(destination_dir)

# # List of base directories to search for .txt files
# # You can add or remove directories as needed
# base_dirs = ['/Users/chengyu/Documents/Corporate Finance/FR from Mcdonald/10-X_C_2011-2015',
#              '/Users/chengyu/Documents/Corporate Finance/FR from Mcdonald/10-X_C_2016-2020',
#              '/Users/chengyu/Documents/Corporate Finance/FR from Mcdonald/2021',
#              '/Users/chengyu/Documents/Corporate Finance/FR from Mcdonald/10-X_C_2022',
#              '/Users/chengyu/Documents/Corporate Finance/FR from Mcdonald/2023'
             
             
#              #'/Users/chengyu/Documents/Corporate Finance/FR from Mcdonald/10-X_C_2006-2010',
#              #'/Users/chengyu/Documents/Corporate Finance/FR from Mcdonald/10-X_C_2001-2005',
#              #'/Users/chengyu/Documents/Corporate Finance/FR from Mcdonald/10-X_C_1993-2000'
#     # Add more directories as needed
# ]

# # Go through each specified directory
# for base_dir in base_dirs:
#     # Walk through the directory
#     for subdir, dirs, files in os.walk(base_dir):
#         for file in files:
#             # Check if the file is a .txt file and does not contain "10Q" or "10-Q"
#             if file.endswith('.txt') and "10Q" not in file and "10-Q" not in file:
#                 # Construct the full file path
#                 file_path = os.path.join(subdir, file)
                
#                 # Construct the destination file path
#                 destination_file_path = os.path.join(destination_dir, file)
                
#                 # Check if the file already exists in the destination folder
#                 if os.path.exists(destination_file_path):
#                     print(f"File {file} already exists in the destination folder. Skipping.")
#                 else:
#                     # Move the file
#                     shutil.move(file_path, destination_file_path)
#                     print(f"Moved {file} to {destination_dir}")

# print("All .txt files without '10Q' and '10-Q' in the name have been moved to", destination_dir)


### STEP2 Extract key sessions form the 10K texts (espcially 7 and 7A)

In [2]:
import pandas as pd
import os
import re

# define extract_info
def extract_info(text, pattern):
    match = re.search(pattern, text)
    return match.group(1).strip() if match else "Not found"

# define extract_section
def extract_section(text, start_heading, end_heading):
    l = len(text)
    start_idx = text.find(start_heading, int(l/5))
    end_idx = text.find(end_heading, start_idx + 1)
    if start_idx != -1 and end_idx != -1:
        return text[start_idx:end_idx].strip()
    else:
        return "Section not found."

# Innitialize the records
records = []

# Assign a directory
path = "/Users/chengyu/Documents/Corporate Finance/10K"  # change the directory if needed

# iterate the texts
for filename in os.listdir(path):
    if filename.endswith(".txt"):
        file_path = os.path.join(path, filename)
        with open(file_path, "r", encoding="utf-8") as file:
            content = file.read()
        
        # establish a dict
        record = {
            "ACCESSION NUMBER": extract_info(content, r"ACCESSION NUMBER:\s+([^\n]+)"),
            "CONFORMED SUBMISSION TYPE": extract_info(content, r"CONFORMED SUBMISSION TYPE:\s+([^\n]+)"),
            "CONFORMED PERIOD OF REPORT": extract_info(content, r"CONFORMED PERIOD OF REPORT:\s+([^\n]+)"),
            "FILED AS OF DATE": extract_info(content, r"FILED AS OF DATE:\s+([^\n]+)"),
            "DATE AS OF CHANGE": extract_info(content, r"DATE AS OF CHANGE:\s+([^\n]+)"),
            "COMPANY CONFORMED NAME": extract_info(content, r"COMPANY CONFORMED NAME:\s+([^\n]+)"),
            "CENTRAL INDEX KEY": extract_info(content, r"CENTRAL INDEX KEY:\s+([^\n]+)"),
            "FISCAL YEAR END": extract_info(content, r"FISCAL YEAR END:\s+([^\n]+)"),
            "Item 7 Content": extract_section(content, "Item 7", "Item 7A"),  
            "Item 7A Content": extract_section(content, "Item 7A", "Item 8")
        }
        
        # append records into the dict
        records.append(record)

# convert records into dataframe
df_records = pd.DataFrame(records)

# print first rows
print(df_records.head())






       ACCESSION NUMBER CONFORMED SUBMISSION TYPE CONFORMED PERIOD OF REPORT  \
0  0000005513-12-000006                      10-K                   20111231   
1  0001171843-17-001588                      10-K                   20161231   
2  0001477932-16-009787                      10-K                   20151231   
3  0001213900-20-028026                      10-K                   20200630   
4  0001144204-13-016260                      10-K                   20121231   

  FILED AS OF DATE DATE AS OF CHANGE        COMPANY CONFORMED NAME  \
0         20120224          20120224                    Unum Group   
1         20170317          20170316        HARVARD BIOSCIENCE INC   
2         20160414          20160414         Thunder Energies Corp   
3         20200923          20200923             SUNHYDROGEN, INC.   
4         20130320          20130320  MEDISAFE 1 TECHNOLOGIES CORP   

  CENTRAL INDEX KEY FISCAL YEAR END  \
0        0000005513            1231   
1        0001123494 

In [3]:
df_records.to_csv('df_records.csv')

In [4]:
# some company files multiple 10-Ks in a year, so need to drop duplicates under the same company/same file year
# Assuming df_records is your DataFrame with the required data

# Step 1: Generate FILE YEAR based on FILED AS OF DATE by extracting the year
df_records['FILE YEAR'] = df_records['FILED AS OF DATE'].astype(str).str[:4]

# Step 2: Convert ACCESSION NUMBER to a comparable format by extracting the numeric part for comparison
df_records['ACCESSION NUMBER INT'] = df_records['ACCESSION NUMBER'].apply(lambda x: int(x.split('-')[-1]))

# Step 3: Sort by FILE YEAR, CENTRAL INDEX KEY, CONFORMED SUBMISSION TYPE, and ACCESSION NUMBER INT (for comparison)
df_records_sorted = df_records.sort_values(by=['FILE YEAR', 'CENTRAL INDEX KEY', 'CONFORMED SUBMISSION TYPE', 'ACCESSION NUMBER INT'], ascending=[True, True, True, False])

# Step 4: Remove duplicates based on FILE YEAR, CENTRAL INDEX KEY, and CONFORMED SUBMISSION TYPE, keeping the first occurrence
df_records_unique = df_records_sorted.drop_duplicates(subset=['FILE YEAR', 'CENTRAL INDEX KEY', 'CONFORMED SUBMISSION TYPE'], keep='first')

# Optional: Remove the auxiliary 'ACCESSION NUMBER INT' column if no longer needed
df_records_unique = df_records_unique.drop(columns=['ACCESSION NUMBER INT'])

# Reset index if necessary
df_records_unique.reset_index(drop=True, inplace=True)

# df_records_unique now holds the processed DataFrame


In [5]:
#quick view over the data
df_records_unique[df_records_unique['FILED AS OF DATE'].astype(float)>20230000].head(10)

,ACCESSION NUMBER,CONFORMED SUBMISSION TYPE,CONFORMED PERIOD OF REPORT,FILED AS OF DATE,DATE AS OF CHANGE,COMPANY CONFORMED NAME,CENTRAL INDEX KEY,FISCAL YEAR END,Item 7 Content,Item 7A Content,FILE YEAR
100169,0001104659-23-082069,10-K,20230531,20230718,20230718,AAR CORP,0000001750,0531,Section not found.,Section not found.,2023
100170,0001628280-23-004026,10-K,20221231,20230217,20230217,ABBOTT LABORATORIES,0000001800,1231,"Item 7, Management s Discussion and Analysis o...","Item 7A, Quantitative and Qualitative Disclosu...",2023
100171,0001264931-23-000006,10-K,20221231,20230331,20230331,WORLDS INC,0000001961,1231,Section not found.,Section not found.,2023
100172,0001564590-23-003422,10-K,20221231,20230310,20230310,ACME UNITED CORP,0000002098,1231,Item 7. Management s Discussion and Analysis o...,Item 7A. Quantitative and Qualitative Disclosu...,2023
100173,0000002178-23-000038,10-K,20221231,20230316,20230316,"ADAMS RESOURCES & ENERGY, INC.",0000002178,1231,Item 7. Management s Discussion and Analysis o...,Item 7A. Quantitative and Qualitative Disclosu...,2023
100174,0001654954-23-003055,10-K,20221231,20230316,20230316,BK Technologies Corp,0000002186,1231,Item 7. Management s Discussion and Analysis o...,Item 7A. Quantitative and Qualitative Disclosu...,2023
100175,0001654954-23-005616,10-K/A,20221231,20230502,20230502,BK Technologies Corp,0000002186,1231,Section not found.,Section not found.,2023
100176,0000002488-23-000047,10-K,20221231,20230227,20230227,ADVANCED MICRO DEVICES INC,0000002488,1230,Section not found.,Section not found.,2023
100177,0000002969-23-000047,10-K,20230930,20231116,20231116,"Air Products & Chemicals, Inc.",0000002969,0930,Item 7. Management's Discussion and Analysis o...,Item 7A. Quantitative and Qualitative Disclosu...,2023
100178,0000950170-23-006169,10-K,20221231,20230306,20230306,CECO ENVIRONMENTAL CORP,0000003197,1231,Section not found.,Section not found.,2023


### STEP3 Intro of Loughran-McD dictionary and prepare TF-IDF, Bag of words method and so on

In [6]:
import pandas as pd
raw_url_dict = 'https://raw.githubusercontent.com/ethancn01/10ksentimentalanalysis/main/Loughran-McDonald_MasterDictionary_1993-2021.csv'
dict = pd.read_csv(raw_url_dict)

In [7]:
#quick view of the dict
columns_to_update = ['Negative', 'Positive', 'Uncertainty', 'Litigious', 'Strong_Modal', 'Weak_Modal', 'Constraining']

for column in columns_to_update:
    dict[column] = dict[column].apply(lambda x: 1 if x != 0 else 0)
dict

,Word,Seq_num,Word Count,Word Proportion,Average Proportion,Std Dev,Doc Count,Negative,Positive,Uncertainty,Litigious,Strong_Modal,Weak_Modal,Constraining,Syllables,Source
0,AARDVARK,1,354,1.550080e-08,1.422600e-08,3.815486e-06,99,0,0,0,0,0,0,0,2,12of12inf
1,AARDVARKS,2,3,1.313627e-10,8.653817e-12,9.241714e-09,1,0,0,0,0,0,0,0,2,12of12inf
2,ABACI,3,9,3.940882e-10,1.169679e-10,5.290465e-08,7,0,0,0,0,0,0,0,3,12of12inf
3,ABACK,4,29,1.269840e-09,6.654735e-10,1.595100e-07,28,0,0,0,0,0,0,0,2,12of12inf
4,ABACUS,5,8570,3.752595e-07,3.809464e-07,3.529356e-05,1108,0,0,0,0,0,0,0,3,12of12inf
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86526,ZYGOTE,86529,50,2.189379e-09,8.729336e-10,1.886011e-07,35,0,0,0,0,0,0,0,2,12of12inf
86527,ZYGOTES,86530,1,4.378757e-11,1.809516e-11,1.932446e-08,1,0,0,0,0,0,0,0,2,12of12inf
86528,ZYGOTIC,86531,0,0.000000e+00,0.000000e+00,0.000000e+00,0,0,0,0,0,0,0,0,3,12of12inf
86529,ZYMURGIES,86532,0,0.000000e+00,0.000000e+00,0.000000e+00,0,0,0,0,0,0,0,0,3,12of12inf


#### A. Bag of Words

In [8]:
#Data Preparation for BoW

df_analysis = df_records_unique.copy()


In [9]:
! pip install dask[complete]  # Install Dask with complete dependencies

zsh:1: no matches found: dask[complete]


In [10]:
import pandas as pd
import dask.dataframe as dd
from dask.diagnostics import ProgressBar
import re
from collections import Counter


/var/folders/nr/053gf3x97yg912g1cs827wrw0000gn/T/ipykernel_1347/3659113706.py:2: DeprecationWarning: The current Dask DataFrame implementation is deprecated. 
In a future release, Dask DataFrame will use new implementation that
contains several improvements including a logical query planning.
The user-facing DataFrame API will remain unchanged.

The new implementation is already available and can be enabled by
installing the dask-expr library:

    $ pip install dask-expr

and turning the query planning option on:

    >>> import dask
    >>> dask.config.set({'dataframe.query-planning': True})
    >>> import dask.dataframe as dd

API documentation for the new implementation is available at
https://docs.dask.org/en/stable/dask-expr-api.html

Any feedback can be reported on the Dask issue tracker
https://github.com/dask/dask/issues 

  import dask.dataframe as dd


In [11]:
df_analysis


,ACCESSION NUMBER,CONFORMED SUBMISSION TYPE,CONFORMED PERIOD OF REPORT,FILED AS OF DATE,DATE AS OF CHANGE,COMPANY CONFORMED NAME,CENTRAL INDEX KEY,FISCAL YEAR END,Item 7 Content,Item 7A Content,FILE YEAR
0,0001047469-11-006302,10-K,20110531,20110713,20110713,AAR CORP,0000001750,0531,Section not found.,Section not found.,2011
1,0001047469-11-001056,10-K,20101231,20110218,20110218,ABBOTT LABORATORIES,0000001800,1231,Section not found.,Section not found.,2011
2,0000950123-11-070238,10-K,20110430,20110729,20110729,"SERVIDYNE, INC.",0000001923,0430,Section not found.,Section not found.,2011
3,0000950123-11-056081,10-K/A,20100430,20110602,20110602,"SERVIDYNE, INC.",0000001923,0430,Section not found.,Section not found.,2011
4,0001264931-11-000102,10-K,20101231,20110309,20110309,"Worlds.com, Inc.",0000001961,0930,Section not found.,Section not found.,2011
...,...,...,...,...,...,...,...,...,...,...,...
108031,0001967649-23-000025,10-K,20230929,20231221,20231221,Vestis Corp,0001967649,0930,Item 7. Management's Discussion and Analysis o...,Item 7A. Quantitative and Qualitative Disclosu...,2023
108032,0001974793-23-000008,10-K,20230930,20231220,20231219,"Oaktree Gardens OLP, LLC",0001974793,1231,Item 7. Management's Discussion and Analysis o...,Item 7A. Quantitative and Qualitative Disclosu...,2023
108033,0001213900-23-097443,10-K,20230930,20231221,20231220,Bloom HoldCo LLC,0001975931,Not found,Section not found.,Section not found.,2023
108034,0001493152-23-046325,10-K,20230930,20231228,20231228,DFP HOLDINGS Ltd,0001976900,0930,Section not found.,Section not found.,2023


In [12]:
import dask.dataframe as dd
import pandas as pd
from dask.diagnostics import ProgressBar


# convert dict into 
dict['Word'] = dict['Word'].str.lower()
sentiment_dict = dict.set_index('Word')[['Negative', 'Positive', 'Uncertainty', 'Litigious', 'Strong_Modal', 'Weak_Modal', 'Constraining']].to_dict('index')

# Use Dask Dataframe to calculate the sentiment result, so we can carry on the analysis with multiple CPU cores simultaneously
ddf_analysis = dd.from_pandas(df_analysis, npartitions=8)

# define sentiment analysis function
def sentiment_analysis(row):
    # first we conmbine item 7 and 7A
    combined_text = (row['Item 7 Content'] + " " + row['Item 7A Content']).lower()
    words = combined_text.split()
    
    # intialize the sentiment counts
    sentiment_counts = {'Negative': 0, 'Positive': 0, 'Uncertainty': 0, 'Litigious': 0, 'Strong_Modal': 0, 'Weak_Modal': 0, 'Constraining': 0}
    
    # count the frequency
    for word in words:
        if word in sentiment_dict:
            for sentiment in sentiment_counts.keys():
                sentiment_counts[sentiment] += sentiment_dict[word].get(sentiment, 0)
                
    return pd.Series(sentiment_counts)

# map the scores into different sentiment columns
meta = {sentiment: 'int' for sentiment in ['Negative', 'Positive', 'Uncertainty', 'Litigious', 'Strong_Modal', 'Weak_Modal', 'Constraining']}
sentiment_columns = ddf_analysis.map_partitions(lambda df: df.apply(sentiment_analysis, axis=1), meta=meta)

# display the progress while running
with ProgressBar():
    computed_sentiments = sentiment_columns.compute()

# convert the result into a dataframe
df_final = df_analysis.join(computed_sentiments)

# display the result
print(df_final.head())



[########################################] | 100% Completed | 335.41 s
       ACCESSION NUMBER CONFORMED SUBMISSION TYPE CONFORMED PERIOD OF REPORT  \
0  0001047469-11-006302                      10-K                   20110531   
1  0001047469-11-001056                      10-K                   20101231   
2  0000950123-11-070238                      10-K                   20110430   
3  0000950123-11-056081                    10-K/A                   20100430   
4  0001264931-11-000102                      10-K                   20101231   

  FILED AS OF DATE DATE AS OF CHANGE COMPANY CONFORMED NAME CENTRAL INDEX KEY  \
0         20110713          20110713               AAR CORP        0000001750   
1         20110218          20110218    ABBOTT LABORATORIES        0000001800   
2         20110729          20110729        SERVIDYNE, INC.        0000001923   
3         20110602          20110602        SERVIDYNE, INC.        0000001923   
4         20110309          20110309       

In [13]:
df_final

,ACCESSION NUMBER,CONFORMED SUBMISSION TYPE,CONFORMED PERIOD OF REPORT,FILED AS OF DATE,DATE AS OF CHANGE,COMPANY CONFORMED NAME,CENTRAL INDEX KEY,FISCAL YEAR END,Item 7 Content,Item 7A Content,FILE YEAR,Negative,Positive,Uncertainty,Litigious,Strong_Modal,Weak_Modal,Constraining
0,0001047469-11-006302,10-K,20110531,20110713,20110713,AAR CORP,0000001750,0531,Section not found.,Section not found.,2011,0,0,0,0,0,0,0
1,0001047469-11-001056,10-K,20101231,20110218,20110218,ABBOTT LABORATORIES,0000001800,1231,Section not found.,Section not found.,2011,0,0,0,0,0,0,0
2,0000950123-11-070238,10-K,20110430,20110729,20110729,"SERVIDYNE, INC.",0000001923,0430,Section not found.,Section not found.,2011,0,0,0,0,0,0,0
3,0000950123-11-056081,10-K/A,20100430,20110602,20110602,"SERVIDYNE, INC.",0000001923,0430,Section not found.,Section not found.,2011,0,0,0,0,0,0,0
4,0001264931-11-000102,10-K,20101231,20110309,20110309,"Worlds.com, Inc.",0000001961,0930,Section not found.,Section not found.,2011,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108031,0001967649-23-000025,10-K,20230929,20231221,20231221,Vestis Corp,0001967649,0930,Item 7. Management's Discussion and Analysis o...,Item 7A. Quantitative and Qualitative Disclosu...,2023,84,34,102,41,15,20,50
108032,0001974793-23-000008,10-K,20230930,20231220,20231219,"Oaktree Gardens OLP, LLC",0001974793,1231,Item 7. Management's Discussion and Analysis o...,Item 7A. Quantitative and Qualitative Disclosu...,2023,41,32,99,16,20,52,35
108033,0001213900-23-097443,10-K,20230930,20231221,20231220,Bloom HoldCo LLC,0001975931,Not found,Section not found.,Section not found.,2023,0,0,0,0,0,0,0
108034,0001493152-23-046325,10-K,20230930,20231228,20231228,DFP HOLDINGS Ltd,0001976900,0930,Section not found.,Section not found.,2023,0,0,0,0,0,0,0


In [14]:
dict.describe()

,Seq_num,Word Count,Word Proportion,Average Proportion,Std Dev,Doc Count,Negative,Positive,Uncertainty,Litigious,Strong_Modal,Weak_Modal,Constraining,Syllables
count,86531.000000,8.653100e+04,8.653100e+04,8.653100e+04,8.653100e+04,8.653100e+04,86531.000000,86531.000000,86531.000000,86531.000000,86531.000000,86531.000000,86531.000000,86531.000000
mean,43267.739007,2.639231e+05,1.155655e-05,1.155585e-05,1.861893e-05,2.104477e+04,0.027216,0.004091,0.003432,0.010459,0.000220,0.000312,0.002126,2.841386
std,24980.758485,8.477175e+06,3.711949e-04,3.736677e-04,1.248076e-04,1.037376e+05,0.162712,0.063830,0.058486,0.101732,0.014817,0.017662,0.046064,1.225919
min,1.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,21633.500000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000
50%,43269.000000,7.700000e+01,3.371643e-09,2.812529e-09,6.047555e-07,4.700000e+01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000
75%,64901.500000,2.291000e+03,1.003173e-07,8.389999e-08,5.536453e-06,9.930000e+02,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.000000
max,86534.000000,1.586171e+09,6.945457e-02,6.984255e-02,1.118392e-02,1.140097e+06,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,9.000000
